# Contando objetos en una imagen con OpenCV

En esta práctica aprenderemos los pasos básicos del proceso de contar objetos en una imagen estática.

La metodología de este ejercicio es parecido a otras prácticas que hemos realizado en el curso: leer la imagen, aplicar algún preprocesamiento, realizar la segmentación de los objetos y, por último, hacer la detección.

Una vez terminada la práctica puede expandir la aplicación para usarla en video, utilizar filtros más avanzados para mejorar el desempeño, etc.

In [1]:
# Importamos las librerías necesarias
import numpy as np
import matplotlib.pyplot as plt
# !pip install opencv-python # en caso de que no tengan instalado opencv
import cv2 as cv

In [2]:
# Comprobamos que tengamos instalado OpenCV
cv.__version__

'4.11.0'

Como estaremos trabajando y experimentando un poco con valores, crearemos una función de Python llamada `countObjects()`

In [3]:
def countObjects(path, minimum_area=100):
    """
    Carga la imagen desde la computadora y cuenta los objetos basados en contornos.

    Parámetros:
    - path --> ruta de la imagen en la computadora (str)
    - minimum_area --> área mínima del contorno (int)
    Regresa:
    - count: número de objetos detectados
    - output: imagen original con los contornos dibujados
    """ 
    # Creamos una variable image que tendrá la dirección a una imagen en la pc
    image = cv.imread(path)

    # Comprobamos que la imagen exista
    if image is None:
        raise FileNotFoundError(f"No se pudo cargar la imagen en '{path}'")
    
    # Convertir la imagen a escala de grises
    gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)

    # Suavizamos un poco la imagen para reducir el ruido en la imagen
    blurred = cv.GaussianBlur(gray, (5,5), 0)

    # El siguiente paso es realizar la umbralización binaria inversa
    # es decir, los objetos más oscuros que el fondo pasaran a blanco (255)
    # Ajustar el valor de acuerdo con la imagen
    _, thresh = cv.threshold(blurred, 60, 255, cv.THRESH_BINARY_INV)

    # Extraer los contornos de la imagen
    contours, _ = cv.findContours(
        thresh.copy(), # creamos una copia de la imagen thresh
        cv.RETR_EXTERNAL,
        cv.CHAIN_APPROX_SIMPLE
    )

    # Dibujar los contornos encontrados y contar
    output = image.copy() # Creamos una copia de la imagen image
    count = 0 # Establecemos el contador en cero
    # Creamos un ciclo for para encontrar y contar todos los contornos
    for cnt in contours:
        area = cv.contourArea(cnt)
        # Si el contorno no pasa el limite mínimo se omite
        if area < minimum_area:
            continue

        count += 1
        # Dibujamos un bounding box para cada objeto encontrado
        x, y, w, h = cv.boundingRect(cnt)
        # Establecer el color verde para el rectangulo
        cv.rectangle(output, (x,y), (x+w, y+h), (0,255,0), 2)
        # Poner etiqeta sobre el objeto
        cv.putText(
            output,
            f'Obj {count}',
            (x, y-10),
            cv.FONT_HERSHEY_SIMPLEX,
            0.5, # tamaño de la fuente
            (0,255,0), # color verde
            2, # grosor de la fuente
        )

    return count, output

Si todo sale bien, cuando ejectuen las siguientes lineas de código, deberían de ver la imagen original con los contornos dibujados en ella, así como los objetos detectados dentro de la imagen.

In [ ]:
# Leer las imagenes dentro de la carpeta
# image = 'coins.jpg'
image = 'clips.jpg'

In [ ]:
# Probar la función con la imagen 1
objetos, img_result = countObjects(image)
print(f"Numero total de objetos detectados: {objetos}")

# Mostrar la imagen con los contornos
cv.imshow("Objetos contados", img_result)
cv.waitKey(0)
cv.destroyAllWindows()

Numero total de objetos detectados: 3


: 

Si experimentan con otro tipo de imágenes u objetos como monedas o piezas de ajedrez comenzaran a notar que el programa no funciona de la mejor manera. En muchos casos, los contornos detectados serán muy pequeños que, aunque el valor de `minimum_area` sea 0, no se podrán encontrar contornos.

Esto se debe a muchos factores como oclusión, solapamiento, contraste de la imagen bajo, el fondo de la imagen, entre otros. Uno de los principales es que usamos un **valor de umbral fijo**. El tener un valor fijo deriva en que el programa no se *adapta* al contexto de la imagen o a las condiciones en las que se tomo la imagen.

En este caso, nosotros trabajamos sobre una imagen por lo que tenemos esa *libertad* de poder ajustarnos a las condiciones a través de experimentar y ajustar los valores de umbralización. Sin embargo, si nos enfrentaramos a un video (una imagen cambiante por así decirlo) no podríamos estar ajustando en cada momento los valores. Actualmente, se han desarrollado nuevas metodologías, distintos tipos de filtros y transformaciónes para llegar a una "solución universal". La IA por ejemplo ha mostrado resultados interesantes y prometedores, pero eso es tema, quizá, para otro curso.